<a href="https://colab.research.google.com/github/chunyulin/tf2020ncku/blob/master/2_ReuseDenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reuse model in Keras Application
* https://keras.io/api/applications/

In [51]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print("X:", x_train.shape)
print("Y:", y_train.shape)

X: (60000, 28, 28, 1)
Y: (60000,)


In [0]:
# Load data with `tf.data.Dataset`

def enlarge(x,y):
  x = tf.reshape(x, (-1,1))
  x = tf.tile(x,  tf.constant([1,3], tf.int32) )
  x = tf.reshape(x, (28,28,3))            ## duplicate 3 channel
  largex = tf.image.resize(x, [32, 32])   ## denseNet need image >= 32x32
  return largex, y

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(enlarge).shuffle(10000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(enlarge).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [53]:
for x,y in test_ds.take(1):
  print(x.shape)


(32, 32, 32, 3)


# Use DenseNet for MNIST (just for fun)
* DenseNet
![DenseNet](https://miro.medium.com/max/1302/1*Cv2IqVWmiakP_boAJODKig.png)
* [Comparasion of models](https://www.cnblogs.com/shine-lee/p/11426134.html)
![Comparasion of models](https://miro.medium.com/max/658/1*XoakexX4n9YSEalWxePqqw.png)
* See also the [original paper of DenseNet](https://arxiv.org/abs/1608.06993)



In [0]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

# Reset the graph to prevent the change of layer's name when re-run this block.
tf.compat.v1.reset_default_graph()

def add_last_layer(base_model, nb_classes):
  """
  Add one layer for model to classify nb_class class
  """
  x = base_model.output
  predict = Dense(nb_classes, activation='softmax')(x)
  return Model(inputs=base_model.input, outputs=predict)

model0 = tf.keras.applications.densenet.DenseNet121(include_top=False, input_shape=(32,32,3))
model = add_last_layer(model0, 10)
#tf.keras.backend.image_data_format()

In [0]:
model.summary()

In [57]:
### You can get the detail of each layer and the weights by its name
layer = model.get_layer('bn')
print(layer.get_config())        ## layer information
print(len(layer.get_weights()))

layer = model.get_layer('conv5_block16_2_conv')
print(layer.get_config())        ## layer information
print(len(layer.get_weights()))


{'name': 'bn', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 1.001e-05, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}}, 'beta_regularizer': None, 'gamma_regularizer': None, 'beta_constraint': None, 'gamma_constraint': None}
4
{'name': 'conv5_block16_2_conv', 'trainable': True, 'dtype': 'float32', 'filters': 32, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint

In [0]:
# Build a new model from the DenseNet and bypassing the last few layers

from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

def modify_model(base_model, layer_name, nb_classes):
  x = base_model.get_layer(layer_name).output
  x = Dense(128, activation='relu')(x)
  predict = Dense(nb_classes, activation=None)(x)
  return Model(inputs=base_model.input, outputs=predict)

model0 = tf.keras.applications.densenet.DenseNet121(include_top=False, input_shape=(32,32,3))
model = modify_model(model0, 'bn', 10)
#tf.keras.backend.image_data_format()

In [0]:
## Define loss and optimzer object
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer   = tf.keras.optimizers.Adam()

## define loss/acc fucntion
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
train_topk_acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='train_topk_acc')

## decorated by tf.function for faster training than TF2 eager mode. See https://www.tensorflow.org/guide/function
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(labels, predictions)
  train_topk_acc(labels, predictions)

In [96]:
EPOCHS = 5
train_loss_history = []

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  train_loss_history.append(np.mean(train_loss.result()))

  template = 'Epoch {}, Loss: {}, Acc: {}, Top-1 Acc: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_acc.result()*100, train_topk_acc.result()*100 ) )

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_acc.reset_states()
  train_topk_acc.reset_states()


Epoch 1, Loss: 0.15893864631652832, Acc: 12.477760314941406, Top-1 Acc: 95.71499633789062
Epoch 2, Loss: 0.11471554636955261, Acc: 12.600313186645508, Top-1 Acc: 97.04166412353516
Epoch 3, Loss: 0.06169906258583069, Acc: 12.687187194824219, Top-1 Acc: 98.25666809082031
Epoch 4, Loss: 0.062211886048316956, Acc: 12.706823348999023, Top-1 Acc: 98.28666687011719
Epoch 5, Loss: 0.06955995410680771, Acc: 12.654374122619629, Top-1 Acc: 98.01333618164062


# Let's make prediction with the model

In [61]:
import matplotlib.pyplot as plt

# Predict 1 sample...
for images,labels in test_ds.take(2):
  print(images.shape)  ## (64, 32, 32, 3)
  im = images[7:8]     ## take 7th image out of the batch.
  print("Predict: ", model.predict(im).argmax())
  #plt.imshow(im.numpy().reshape(32,32))
  #plt.show()


(32, 32, 32, 3)
Predict:  9
(32, 32, 32, 3)
Predict:  1


In [0]:
# Predict multimple samples
for images,labels in test_ds.take(1):
  #print(images.shape)                # (32, 32, 32, 3)
  #print(model.predict(images).shape) # (32, 1, 1, 10)
  pre = model.predict(images).argmax(axis=-1).flatten()
  print( pre - labels.numpy() )
  #for i in range(32):
  #  plt.figure()
  #  plt.imshow(images[i])
  #  plt.title("Label:%d Pred:%d" % (labels[i], pre[i]))


### That is all for today. For more `tf.keras` usage, see some tricks in [Keras FAQ](https://keras.io/getting_started/faq/).